In [1]:
import numpy as np
import pandas as pd

In [2]:
!wget https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3937nnn/GSM3937266/suppl/GSM3937266_FC_phRNAi.txt.gz -q
!gunzip GSM3937266_FC_phRNAi.txt.gz -f
!wget ftp://ftp.flybase.net/genomes//Drosophila_melanogaster/dmel_r6.48_FB2022_05/gtf/dmel-all-r6.48.gtf.gz -q
!gunzip dmel-all-r6.48.gtf.gz -f

In [3]:
rna = pd.read_csv("GSM3937266_FC_phRNAi.txt", sep="\t")

In [4]:
rna["quartiles"] = 0

In [5]:
rna.loc[rna["baseMean"] <= np.quantile(rna["baseMean"], 0.25), "quartile"] = 1
rna.loc[rna["baseMean"] > np.quantile(rna["baseMean"], 0.25), "quartile"] = 2
rna.loc[rna["baseMean"] > np.quantile(rna["baseMean"], 0.5), "quartile"] = 3
rna.loc[rna["baseMean"] > np.quantile(rna["baseMean"], 0.75), "quartile"] = 4

In [6]:
!awk '{print $1,$4,$5,$7,$10,$3}' dmel-all-r6.48.gtf | \
awk '{gsub(";",""); print}' | awk '{gsub(/"/, ""); print}' | \
awk -v OFS="\t" '{if($6=="gene") {print $1,$2,$3,$4,$5}}' \
> dm6_genes.txt

In [7]:
genes = pd.read_csv("dm6_genes.txt", sep="\t", header=None)
genes.columns = ["chrom", "start", "end", "strand", "FBgn"]

In [8]:
merge = pd.merge(rna, genes, on="FBgn", how="left")

In [9]:
merge["start1"] = 0
merge.loc[merge["strand"] == "+", "start1"] = merge.loc[merge["strand"] == "+", "start"]
merge.loc[merge["strand"] == "-", "start1"] = merge.loc[merge["strand"] == "-", "end"]
merge["end1"] = merge["start1"]+1
merge["chrom"] = "chr" + merge["chrom"]

In [10]:
for i in range(1,5):
    merge.loc[merge["quartile"] == i, 
              ["chrom", "start1", "end1"]].to_csv("Loubiere_RNAseq_quartile_"+str(i)+"_dm6.bed",
                                                  sep="\t", header=None, index=False)

In [11]:
!wget ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1910nnn/GSM1910622/suppl/GSM1910622_EyeDisc_K27Ac_Enriched.bed.gz -q
!gunzip GSM1910622_EyeDisc_K27Ac_Enriched.bed.gz -f
!wget https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3619nnn/GSM3619410/suppl/GSM3619410_ChIP_PSC_ED_merge_peaks.narrowPeak.gz -q
!gunzip GSM3619410_ChIP_PSC_ED_merge_peaks.narrowPeak.gz -f
!wget https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3619nnn/GSM3619412/suppl/GSM3619412_ChIP_SUZ12_ED_merge_peaks.narrowPeak.gz -q
!gunzip GSM3619412_ChIP_SUZ12_ED_merge_peaks.narrowPeak.gz -f

In [12]:
!awk -v OFS="\t" '{print "chr"$1,$2,$3}' GSM1910622_EyeDisc_K27Ac_Enriched.bed \
> GSM1910622_EyeDisc_K27Ac_Enriched_chr.bed

In [13]:
!$elias/seqtools/./liftOver GSM1910622_EyeDisc_K27Ac_Enriched_chr.bed $elias/seqtools/dm3ToDm6.over.chain.gz \
ED_H3K27ac_dm6.bed unmapped

Reading liftover chains
Mapping coordinates


In [14]:
!cat GSM3619410_ChIP_PSC_ED_merge_peaks.narrowPeak GSM3619412_ChIP_SUZ12_ED_merge_peaks.narrowPeak \
| bedtools sort -i | bedtools merge > ED_PRC1_dm6.bed

In [15]:
!bedtools intersect -v -a ED_H3K27ac_dm6.bed -b ED_PRC1_dm6.bed \
> ED_H3K27ac_noPRC1_dm6.bed

In [16]:
!bedtools intersect -a ED_H3K27ac_dm6.bed -b ED_PRC1_dm6.bed -wa \
> ED_H3K27ac_PRC1_dm6.bed

In [1]:
!bedtools intersect -v -b ED_H3K27ac_dm6.bed -a ED_PRC1_dm6.bed -wa \
> ED_PRC1_noH3K27ac_dm6.bed

In [18]:
import cooler

In [19]:
clr = cooler.Cooler("ED.allValidPairs.mcool::/resolutions/5000")

In [20]:
dic={chr: "chr"+str(chr) for chr in clr.chromnames}

In [21]:
cooler.rename_chroms(clr, dic)

/exports/igmm/eddie/chr11/elias/anaconda/envs/coolpuppy_trans/lib/python3.8/site-packages/cooler/create/_create.py:353: FutureWarning: is_categorical is deprecated and will be removed in a future version. Use is_categorical_dtype instead.
  if is_categorical(bins["chrom"]) or is_integer(bins["chrom"]):
